In [ ]:
# default_exp setup

# Setup functions
> setup details here.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# !pip install -Uqq kaggle
# !pip install -Uqq wandb
# !pip install -Uqq python-dotenv

In [ ]:
from fastai.vision.all import *
from kaggle.api.kaggle_api_extended import KaggleApi
import wandb
from dotenv import load_dotenv
import os
from tqdm import tqdm

load_dotenv()
k = KaggleApi()
k.authenticate()

In [ ]:
#export 
def get_kaggle_dataset(competition):
    'download and extract kaggle competition dataset to fastai data folder'
    config = Config()
    zipdest = Path(config.d['archive_path'])
    zipname = Path(config.d['archive_path']+'/'+competition+'.zip')
    if not zipname.exists():
        print (f'${zipname} does not exist. Downloading...')
        k.competition_download_files(competition, path=zipdest) 
    dest = Path(config.d['data_path']+'/'+competition+'/')
    if not dest.exists():
        print (f'${dest} does not exist.  Extracting...')
        file_extract(zipname, dest=dest)
        files = os.listdir(dest)
        for f in files:
            if f.split('.')[-1]=='zip':
                print(dest/f)
                file_extract(dest/f)
                os.remove(dest/f)
    return dest

In [ ]:
k.competitions_list(search='dogs')

[dogs-vs-cats, dogs-vs-cats-redux-kernels-edition]

In [ ]:
p = get_kaggle_dataset('dogs-vs-cats-redux-kernels-edition')
p

Path('/home/fredguth/.fastai/data/dogs-vs-cats-redux-kernels-edition')

In [ ]:
# export
def download_and_log(competition):
    # 🚀 start a run, with a type to label it and a project it can call home
    with wandb.init(project=competition, job_type="download-data") as run:
        path = get_kaggle_dataset(competition)
        sizes = {}
        raw_data = wandb.Artifact(
            competition, type="dataset",
            description=f'Raw {competition} dataset.',
            metadata={"source": "Kaggle"})
        for b, ds, _ in os.walk(path):
            for d in ds:
                files = get_image_files(b+'/'+d)
                sizes[d] = len(files)
                for f in tqdm(files):
                    raw_data.add_reference('file://'+str(f))
        raw_data.metadata['sizes']=sizes
        # ✍️ Save the artifact to W&B.
        run.log_artifact(raw_data)

In [ ]:
download_and_log('dogs-vs-cats-redux-kernels-edition')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fredguth (use `wandb login --relogin` to force relogin)


100%|██████████| 25000/25000 [00:06<00:00, 4070.46it/s]
